In [1]:
import os
import sys
import time
import modin.pandas as pd
from modin.experimental.core.execution.native.implementations.omnisci_on_native.omnisci_worker import OmnisciServer


def read(filename):
    columns_names = [
        "trip_id",
        "vendor_id",
        "pickup_datetime",
        "dropoff_datetime",
        "store_and_fwd_flag",
        "rate_code_id",
        "pickup_longitude",
        "pickup_latitude",
        "dropoff_longitude",
        "dropoff_latitude",
        "passenger_count",
        "trip_distance",
        "fare_amount",
        "extra",
        "mta_tax",
        "tip_amount",
        "tolls_amount",
        "ehail_fee",
        "improvement_surcharge",
        "total_amount",
        "payment_type",
        "trip_type",
        "pickup",
        "dropoff",
        "cab_type",
        "precipitation",
        "snow_depth",
        "snowfall",
        "max_temperature",
        "min_temperature",
        "average_wind_speed",
        "pickup_nyct2010_gid",
        "pickup_ctlabel",
        "pickup_borocode",
        "pickup_boroname",
        "pickup_ct2010",
        "pickup_boroct2010",
        "pickup_cdeligibil",
        "pickup_ntacode",
        "pickup_ntaname",
        "pickup_puma",
        "dropoff_nyct2010_gid",
        "dropoff_ctlabel",
        "dropoff_borocode",
        "dropoff_boroname",
        "dropoff_ct2010",
        "dropoff_boroct2010",
        "dropoff_cdeligibil",
        "dropoff_ntacode",
        "dropoff_ntaname",
        "dropoff_puma",
    ]
    # use string instead of category
    columns_types = [
        "int64",
        "string",
        "timestamp",
        "timestamp",
        "string",
        "int64",
        "float64",
        "float64",
        "float64",
        "float64",
        "int64",
        "float64",
        "float64",
        "float64",
        "float64",
        "float64",
        "float64",
        "float64",
        "float64",
        "float64",
        "string",
        "float64",
        "string",
        "string",
        "string",
        "float64",
        "int64",
        "float64",
        "int64",
        "int64",
        "float64",
        "float64",
        "float64",
        "float64",
        "string",
        "float64",
        "float64",
        "string",
        "string",
        "string",
        "float64",
        "float64",
        "float64",
        "float64",
        "string",
        "float64",
        "float64",
        "string",
        "string",
        "string",
        "float64",
    ]

    dtypes = {columns_names[i]: columns_types[i] for i in range(len(columns_names))}
    all_but_dates = {
        col: valtype
        for (col, valtype) in dtypes.items()
        if valtype not in ["timestamp"]
    }
    dates_only = [col for (col, valtype) in dtypes.items() if valtype in ["timestamp"]]

    df = pd.read_csv(
        filename,
        names=columns_names,
        dtype=all_but_dates,
        parse_dates=dates_only,
    )

    df.shape  # to trigger real execution
    df._query_compiler._modin_frame._partitions[0][
        0
    ].frame_id = OmnisciServer().put_arrow_to_omnisci(
        df._query_compiler._modin_frame._partitions[0][0].get()
    )  # to trigger real execution
    return df


def q1_omnisci(df):
    q1_pandas_output = df.groupby("cab_type").size()
    q1_pandas_output.shape  # to trigger real execution
    return q1_pandas_output


def q2_omnisci(df):
    q2_pandas_output = df.groupby("passenger_count").agg({"total_amount": "mean"})
    q2_pandas_output.shape  # to trigger real execution
    return q2_pandas_output


def q3_omnisci(df):
    df["pickup_datetime"] = df["pickup_datetime"].dt.year
    q3_pandas_output = df.groupby(["passenger_count", "pickup_datetime"]).size()
    q3_pandas_output.shape  # to trigger real execution
    return q3_pandas_output


def q4_omnisci(df):
    df["pickup_datetime"] = df["pickup_datetime"].dt.year
    df["trip_distance"] = df["trip_distance"].astype("int64")
    q4_pandas_output = (
        df.groupby(["passenger_count", "pickup_datetime", "trip_distance"], sort=False)
        .size()
        .reset_index()
        .sort_values(
            by=["pickup_datetime", 0], ignore_index=True, ascending=[True, False]
        )
    )
    q4_pandas_output.shape  # to trigger real execution
    return q4_pandas_output


def measure(name, func, *args, **kw):
    t0 = time.time()
    res = func(*args, **kw)
    t1 = time.time()
    print(f"{name}: {t1 - t0} sec")
    return res

In [2]:
pd.DataFrame({1: [2, 3]})

,1
0,2
1,3


In [3]:
df = measure('Reading', read, '/data/taxi/trips_xaa.csv')

Thrift: Thu Aug  5 22:24:27 2021 TSocket::open() error on socket (after THRIFT_POLL) <Host: localhost Port: 3279>: Connection refused


Reading: 32.167804479599 sec


Thrift: Thu Aug  5 22:24:27 2021 TSocket::open() error on socket (after THRIFT_POLL) <Host: localhost Port: 3279>: Connection refused
Thrift: Thu Aug  5 22:24:28 2021 TSocket::open() error on socket (after THRIFT_POLL) <Host: localhost Port: 3279>: Connection refused
Thrift: Thu Aug  5 22:24:28 2021 TSocket::open() error on socket (after THRIFT_POLL) <Host: localhost Port: 3279>: Connection refused
Thrift: Thu Aug  5 22:24:28 2021 TSocket::open() error on socket (after THRIFT_POLL) <Host: localhost Port: 3279>: Connection refused
Thrift: Thu Aug  5 22:24:28 2021 TSocket::open() error on socket (after THRIFT_POLL) <Host: localhost Port: 3279>: Connection refused
Thrift: Thu Aug  5 22:24:28 2021 TSocket::open() error on socket (after THRIFT_POLL) <Host: localhost Port: 3279>: Connection refused
Thrift: Thu Aug  5 22:24:28 2021 TSocket::open() error on socket (after THRIFT_POLL) <Host: localhost Port: 3279>: Connection refused
Thrift: Thu Aug  5 22:24:28 2021 TSocket::open() error on sock

In [4]:
measure('Q1', q1_omnisci, df)

Q1: 0.12713217735290527 sec


cab_type
green    20000000
dtype: int64

In [5]:
measure('Q2', q2_omnisci, df)

Q2: 0.23839879035949707 sec


,total_amount
passenger_count,
0,9.841277
1,14.417759
2,15.336595
3,15.534873
4,15.496616
5,14.466787
6,15.234298
7,20.582484
8,16.502798


In [6]:
measure('Q3', q3_omnisci, df.copy())

Q3: 0.17857718467712402 sec


passenger_count  pickup_datetime
0                2013                    441
                 2014                   3346
                 2015                    960
1                2013                 958867
                 2014               13112797
                 2015                2482766
2                2013                  93204
                 2014                1162332
                 2015                 212146
3                2013                  27774
                 2014                 377361
                 2015                  67528
4                2013                  11356
                 2014                 126728
                 2015                  19023
5                2013                 104758
                 2014                 781022
                 2015                 122513
6                2013                  14346
                 2014                 272599
                 2015                  47056
7                2013 

In [7]:
measure('Q4', q4_omnisci, df.copy())

Q4: 0.4534273147583008 sec


,passenger_count,pickup_datetime,trip_distance,0
0,1,2015,83,1
1,2,2015,62,1
2,3,2015,85,1
3,1,2015,86,1
4,1,2015,94,1
5,1,2015,96,1
6,1,2015,97,1
7,2,2015,99,1
8,2,2015,100,1
9,1,2015,104,1
